In [2]:
import os.path as osp
import cv2
import numpy as np

import sys
sys.path.append('../../../') 

from dust3r.datasets.base.base_stereo_view_dataset import BaseStereoViewDataset
from dust3r.utils.image import imread_cv2
from dust3r.datasets import UnderWaterDataset

from dust3r.datasets.base.base_stereo_view_dataset import view_name
from dust3r.viz import SceneViz, auto_cam_size
from dust3r.utils.image import rgb

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
dataset = UnderWaterDataset(split='train', ROOT='/home/aleksandra/dense_glomap_output', resolution=224)

print(len(dataset.split_data))
selected_pairs = dataset.selected_pairs
print(len(selected_pairs))

views = dataset[0]
view = views[1]

100%|██████████| 19/19 [00:05<00:00,  3.37it/s]

LEN selected_pairs 0
249
1


In [4]:
valid_points = np.copy(view["pts3d"][view["valid_mask"]])
valid_points = valid_points.astype(np.float64)

print(valid_points.shape)
print(valid_points.dtype)

(43670, 3)
float64


In [17]:
# points_new = np.array(valid_points[:10, ])
# print(points_new.shape)
# print(points_new.dtype)
# print(points_new)
# points_new = points_new.astype(np.float64)
# valid_points = valid_points.astype(np.float64)

(10, 3)
float64
[[ -6.31342793   5.82752228 -20.76261139]
 [ -6.38282871   6.23450851 -21.00959396]
 [ -6.30468273   6.26910782 -21.05754662]
 [-10.69704151  21.76908302 -29.90265274]
 [-10.43704224  21.3627243  -29.71722221]
 [-10.74254799  22.85458755 -30.60616684]
 [-10.88243771  23.20835876 -30.79533386]
 [ -8.65737915  21.21623993 -30.19113922]
 [ -8.81993771  21.58157921 -30.38068581]
 [ -8.77456188  21.60211563 -30.40882683]]


In [5]:
import open3d as o3d
import time

o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)

In [6]:
import numpy as np

def map_colors_to_points(view, rotation_angle=0):
    intrinsics = view["camera_intrinsics"]
    # print("Camera Intrinsics (K):\n", intrinsics)
    
    # Extract extrinsics (camera pose)
    extrinsics = view["camera_pose"]  # camout2world matrix
    # print("Camera Extrinsics (Pose - camout2world):\n", extrinsics)
    
    # Extract and filter point cloud
    points_3d = view["pts3d"]
    valid_mask = view["valid_mask"]
    valid_points = points_3d[valid_mask]
    # print(f"Number of valid points in point cloud: {valid_points.shape[0]}")
    
    # Rotate point cloud around Z-axis
    rotation_matrix = np.array([
        [np.cos(rotation_angle), -np.sin(rotation_angle), 0],
        [np.sin(rotation_angle), np.cos(rotation_angle), 0],
        [0, 0, 1]
    ])
    rotated_points = (rotation_matrix @ valid_points.T).T  # Apply rotation
    
    # Normalize the point cloud for coloring
    z_min, z_max = rotated_points[:, 2].min(), rotated_points[:, 2].max()
    normalized_z = (rotated_points[:, 2] - z_min) / (z_max - z_min)

    return normalized_z


In [7]:
def save_point_cloud_with_colors(filename, points, colors):
    
    if points.ndim != 2 or points.shape[1] != 3:
        raise ValueError("The points array must have shape (N, 3).")
    
    if colors.ndim != 2 or colors.shape[1] != 3:
        raise ValueError("The colors array must have shape (N, 3).")
    
    if points.shape[0] != colors.shape[0]:
        raise ValueError("The number of points and colors must match.")
    
    # Create the point cloud
    start_time = time.time()
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points)
    point_cloud.colors = o3d.utility.Vector3dVector(colors)  # Add colors
    o3d.io.write_point_cloud(filename, point_cloud)
    end_time = time.time()
    
    print(f"Time taken: {end_time - start_time:.6f} seconds")
    print(f"Point cloud with colors saved to {filename}")

In [5]:
# def save_point_cloud_to_ply(filename, points):
#     print(points.shape)
#     if points.ndim != 2 or points.shape[1] != 3:
#         raise ValueError("The points array must have shape (N, 3).")
#     
#     start_time = time.time()
#     point_cloud = o3d.geometry.PointCloud()
#     point_cloud.points = o3d.utility.Vector3dVector(points)
#     end_time = time.time()
#     
#     print(f"Time taken: {end_time - start_time:.6f} seconds")
#     
#     # Comment the saving step temporarily:
#     o3d.io.write_point_cloud(filename, point_cloud)
#     print(f"Point cloud saved to {filename}")
#     
# # test_points = np.array([[0, 0, 0], [1, 1, 1], [2, 2, 2]], dtype=np.float64)
# # save_point_cloud_to_ply("test_output.ply", test_points)


In [17]:
colors = map_colors_to_points(view)

In [18]:
print(colors.shape)
print(colors.dtype)

(43670,)
float64


In [19]:
grayscale_normalized = colors / 255.0

# Define RGB colors for blue (dark) and red (light)
blue = np.array([0, 0, 1])  # RGB for blue
red = np.array([1, 0, 0])   # RGB for red

# Interpolate between blue and red
colors = np.outer(1 - grayscale_normalized, blue) + np.outer(grayscale_normalized, red)


# colors = np.stack([colors] * 3, axis=1)
print(colors.shape)
print(colors.dtype)

(43670, 3)
float64


In [20]:
save_point_cloud_with_colors("/home/aleksandra/coral/models/dust3r/clouds/output_c1.ply", valid_points, colors)

[Open3D DEBUG] Write geometry::PointCloud: 43670 vertices.
Time taken: 0.008986 seconds
Point cloud with colors saved to /home/aleksandra/coral/models/dust3r/clouds/output_c1.ply


In [ ]:
print(valid_points.dtype)

In [ ]:
valid_points

In [ ]:
visualize_intrinsics_extrinsics_pointcloud(views[1], 90)

In [ ]:
import pptk
import numpy as np
P = np.random.rand(100,3)
v = pptk.viewer(P)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_intrinsics_extrinsics_pointcloud(view, rotation_angle=30):
    # Extract intrinsics
    intrinsics = view["camera_intrinsics"]
    # print("Camera Intrinsics (K):\n", intrinsics)
    
    # Extract extrinsics (camera pose)
    extrinsics = view["camera_pose"]  # camout2world matrix
    # print("Camera Extrinsics (Pose - camout2world):\n", extrinsics)
    
    # Extract and filter point cloud
    points_3d = view["pts3d"]
    valid_mask = view["valid_mask"]
    valid_points = points_3d[valid_mask]
    # print(f"Number of valid points in point cloud: {valid_points.shape[0]}")
    
    # Rotate point cloud around Z-axis
    rotation_matrix = np.array([
        [np.cos(rotation_angle), -np.sin(rotation_angle), 0],
        [np.sin(rotation_angle), np.cos(rotation_angle), 0],
        [0, 0, 1]
    ])
    rotated_points = (rotation_matrix @ valid_points.T).T  # Apply rotation
    
    # Normalize the point cloud for coloring
    z_min, z_max = rotated_points[:, 2].min(), rotated_points[:, 2].max()
    normalized_z = (rotated_points[:, 2] - z_min) / (z_max - z_min)
    
    # Plot point cloud in 3D
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection="3d")
    
    # Scatter plot of 3D points
    scatter = ax.scatter(
        rotated_points[:, 0], rotated_points[:, 1], rotated_points[:, 2],
        s=5, c=normalized_z, cmap='plasma', marker='o'
    )
    ax.set_title("3D Point Cloud (Rotated)", fontsize=14)
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_zlabel("Z-axis")
    
    # Adjust axes limits to zoom in
    buffer = 0.1  # Add a small buffer around the points
    ax.set_xlim(rotated_points[:, 0].min() - buffer, rotated_points[:, 0].max() + buffer)
    ax.set_ylim(rotated_points[:, 1].min() - buffer, rotated_points[:, 1].max() + buffer)
    ax.set_zlim(rotated_points[:, 2].min() - buffer, rotated_points[:, 2].max() + buffer)
    
    # Add a colorbar to indicate depth
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=10)
    cbar.set_label('Depth (Z-axis)', fontsize=12)
    
    plt.show()
    
    # Plot camera frustum (optional)
    draw_camera_frustum(ax, intrinsics, extrinsics)

def draw_camera_frustum(ax, intrinsics, extrinsics, scale=0.1):
    # Camera intrinsics: fx, fy, cx, cy
    fx, fy = intrinsics[0, 0], intrinsics[1, 1]
    cx, cy = intrinsics[0, 2], intrinsics[1, 2]
    
    # Image corners in pixel coordinates (assuming normalized image plane)
    image_corners = np.array([
        [-cx / fx, -cy / fy, 1],   # Top-left
        [(1 - cx) / fx, -cy / fy, 1],   # Top-right
        [(1 - cx) / fx, (1 - cy) / fy, 1],  # Bottom-right
        [-cx / fx, (1 - cy) / fy, 1]   # Bottom-left
    ]) * scale  # Scale the size
    
    # Transform image corners to world coordinates using extrinsics
    camera_origin = extrinsics[:3, 3]  # Camera origin in world coordinates
    rotation = extrinsics[:3, :3]  # Camera rotation matrix
    world_corners = (rotation @ image_corners.T).T + camera_origin

    # Draw the frustum
    for i, corner in enumerate(world_corners):
        ax.plot(
            [camera_origin[0], corner[0]],
            [camera_origin[1], corner[1]],
            [camera_origin[2], corner[2]],
            'r--'
        )
    
    # Connect the corners to form the frustum edges
    edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
    for i, j in edges:
        ax.plot(
            [world_corners[i, 0], world_corners[j, 0]],
            [world_corners[i, 1], world_corners[j, 1]],
            [world_corners[i, 2], world_corners[j, 2]],
            'r-'
        )


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_intrinsics_extrinsics_pointcloud(view):
    # Extract intrinsics
    intrinsics = view["camera_intrinsics"]
    print("Camera Intrinsics (K):\n", intrinsics)
    
    # Extract extrinsics (camera pose)
    extrinsics = view["camera_pose"]
    print("Camera Extrinsics (Pose):\n", extrinsics)
    
    # Extract and filter point cloud
    points_3d = view["pts3d"]
    valid_mask = view["valid_mask"]
    valid_points = points_3d[valid_mask]
    print(f"Number of valid points in point cloud: {valid_points.shape[0]}")
    
    # Normalize the point cloud for coloring
    z_min, z_max = valid_points[:, 2].min(), valid_points[:, 2].max()
    normalized_z = (valid_points[:, 2] - z_min) / (z_max - z_min)
    
    # Plot point cloud in 3D
    fig = plt.figure(figsize=(40, 20))
    ax = fig.add_subplot(111, projection="3d")
    
    # Scatter plot of 3D points
    scatter = ax.scatter(
        valid_points[:, 0], valid_points[:, 1], valid_points[:, 2],
        s=5, c=normalized_z, cmap='plasma', marker='o'
    )
    ax.set_title("3D Point Cloud", fontsize=14)
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_zlabel("Z-axis")
    
    # Adjust axes limits to zoom in
    buffer = 0.1  # Add a small buffer around the points
    ax.set_xlim(valid_points[:, 0].min() - buffer, valid_points[:, 0].max() + buffer)
    ax.set_ylim(valid_points[:, 1].min() - buffer, valid_points[:, 1].max() + buffer)
    ax.set_zlim(valid_points[:, 2].min() - buffer, valid_points[:, 2].max() + buffer)
    
    # Add a colorbar to indicate depth
    cbar = plt.colorbar(scatter, ax=ax, shrink=0.5, aspect=10)
    cbar.set_label('Depth (Z-axis)', fontsize=12)
    
    plt.show()
    
    # Plot camera frustum (optional)
    draw_camera_frustum(ax, intrinsics, extrinsics)

def draw_camera_frustum(ax, intrinsics, extrinsics, scale=0.1):
    """
    Draws the camera frustum in the 3D space.
    """
    # Camera intrinsics: fx, fy, cx, cy
    fx, fy = intrinsics[0, 0], intrinsics[1, 1]
    cx, cy = intrinsics[0, 2], intrinsics[1, 2]
    
    # Image corners in pixel coordinates (assuming normalized image plane)
    image_corners = np.array([
        [-cx / fx, -cy / fy, 1],   # Top-left
        [(1 - cx) / fx, -cy / fy, 1],   # Top-right
        [(1 - cx) / fx, (1 - cy) / fy, 1],  # Bottom-right
        [-cx / fx, (1 - cy) / fy, 1]   # Bottom-left
    ]) * scale  # Scale the size
    
    # Transform image corners to world coordinates using extrinsics
    camera_origin = extrinsics[:3, 3]  # Camera origin in world coordinates
    rotation = extrinsics[:3, :3]  # Camera rotation matrix
    world_corners = (rotation @ image_corners.T).T + camera_origin

    # Draw the frustum
    for i, corner in enumerate(world_corners):
        ax.plot(
            [camera_origin[0], corner[0]],
            [camera_origin[1], corner[1]],
            [camera_origin[2], corner[2]],
            'r--'
        )
    
    # Connect the corners to form the frustum edges
    edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
    for i, j in edges:
        ax.plot(
            [world_corners[i, 0], world_corners[j, 0]],
            [world_corners[i, 1], world_corners[j, 1]],
            [world_corners[i, 2], world_corners[j, 2]],
            'r-'
        )


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def visualize_intrinsics_extrinsics_pointcloud(view):
    # Extract intrinsics
    intrinsics = view["camera_intrinsics"]
    print("Camera Intrinsics (K):\n", intrinsics)
    
    # Extract extrinsics (camera pose)
    extrinsics = view["camera_pose"]
    print("Camera Extrinsics (Pose):\n", extrinsics)
    
    # Extract and filter point cloud
    points_3d = view["pts3d"]
    valid_mask = view["valid_mask"]
    valid_points = points_3d[valid_mask]
    print(f"Number of valid points in point cloud: {valid_points.shape[0]}")
    
    # Plot point cloud in 3D
    fig = plt.figure(figsize=(30, 10))
    ax = fig.add_subplot(111, projection="3d")
    
    # Scatter plot of 3D points
    ax.scatter(valid_points[:, 0], valid_points[:, 1], valid_points[:, 2],
               s=0.1, c=valid_points[:, 2], cmap='viridis', marker='o')
    ax.set_title("3D Point Cloud")
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_zlabel("Z-axis")
    plt.show()
    
    # Plot camera frustum (optional)
    draw_camera_frustum(ax, intrinsics, extrinsics)

def draw_camera_frustum(ax, intrinsics, extrinsics, scale=0.1):
    """
    Draws the camera frustum in the 3D space.
    """
    # Camera intrinsics: fx, fy, cx, cy
    fx, fy = intrinsics[0, 0], intrinsics[1, 1]
    cx, cy = intrinsics[0, 2], intrinsics[1, 2]
    
    # Image corners in pixel coordinates (assuming normalized image plane)
    image_corners = np.array([
        [-cx / fx, -cy / fy, 1],   # Top-left
        [(1 - cx) / fx, -cy / fy, 1],   # Top-right
        [(1 - cx) / fx, (1 - cy) / fy, 1],  # Bottom-right
        [-cx / fx, (1 - cy) / fy, 1]   # Bottom-left
    ]) * scale  # Scale the size
    
    # Transform image corners to world coordinates using extrinsics
    camera_origin = extrinsics[:3, 3]  # Camera origin in world coordinates
    rotation = extrinsics[:3, :3]  # Camera rotation matrix
    world_corners = (rotation @ image_corners.T).T + camera_origin

    # Draw the frustum
    for i, corner in enumerate(world_corners):
        ax.plot(
            [camera_origin[0], corner[0]],
            [camera_origin[1], corner[1]],
            [camera_origin[2], corner[2]],
            'r--'
        )
    
    # Connect the corners to form the frustum edges
    edges = [(0, 1), (1, 2), (2, 3), (3, 0)]
    for i, j in edges:
        ax.plot(
            [world_corners[i, 0], world_corners[j, 0]],
            [world_corners[i, 1], world_corners[j, 1]],
            [world_corners[i, 2], world_corners[j, 2]],
            'r-'
        )

In [ ]:
views = dataset[0]  # Fetch the first element

for view in views:
    print(view["instance"])
    print("Image shape:", view["img"].shape)
    print("Depth map shape:", view["depthmap"].shape)
    print("Camera intrinsics:\n", view["camera_intrinsics"])
    print("Camera pose:\n", view["camera_pose"])

    # Visualize depth map
    import matplotlib.pyplot as plt
    plt.imshow(view["depthmap"])
    plt.title("Depth Map")
    plt.show()

    # Check 3D points
    pts3d = view["pts3d"]
    print("3D points shape:", pts3d.shape)
    print("Valid mask:", view["valid_mask"].sum(), "valid points")

In [ ]:
view = views[0]
pcd = o3d.geometry.PointCloud()

In [ ]:
pointcloud1 = pointcloud1[np.random.choice(len(pointcloud1), pixels_count, replace=False)]

In [ ]:
pointcloud1 = view["pts3d"][view["valid_mask"]]

In [ ]:
pointcloud1 = pointcloud1[np.random.choice(len(pointcloud1), 10, replace=False)]

In [ ]:
pcd.points = o3d.utility.Vector3dVector(pointcloud1)

In [ ]:
a = np.array([[1, 2, 3],
             [4, 5, 6], 
             [7, 8, 9]])
    
a[:2, 2].round().astype(int)

In [ ]:
# from tqdm import tqdm
# 
# valid_points = view["pts3d"][view["valid_mask"]].astype(np.float64)  # Ensure dtype is float64
# 
# # Create the point cloud
# pcd = o3d.geometry.PointCloud()
# 
# # Use tqdm to monitor progress (if needed for other tasks, but avoid looping per point for Open3D)
# for _ in tqdm(range(1), desc="Processing Points", unit="points"):
#     pcd.points = o3d.utility.Vector3dVector(valid_points)  # Assign all points at once

In [ ]:
o3d.io.write_point_cloud("pointcloud.ply", pcd)
print("Point cloud saved to 'pointcloud.ply'")

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(view["pts3d"][view["valid_mask"]])

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
def visualize_cameras_and_pointcloud(camera_extrinsics, intrinsics, point_cloud):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Create Open3D point cloud
    pc = o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(point_cloud)
    vis.add_geometry(pc)

    # Visualize cameras
    for extrinsic in camera_extrinsics:
        cam_frame = o3d.geometry.LineSet.create_camera_visualization(
            intrinsics.width, intrinsics.height, intrinsics.intrinsic_matrix, extrinsic, scale=0.1
        )
        vis.add_geometry(cam_frame)

    vis.run()
    vis.destroy_window()

def reproject_points(points_3d, intrinsics):
    fx, fy, cx, cy = intrinsics["fx"], intrinsics["fy"], intrinsics["cx"], intrinsics["cy"]
    u = fx * points_3d[:, 0] / points_3d[:, 2] + cx
    v = fy * points_3d[:, 1] / points_3d[:, 2] + cy
    return np.stack([u, v], axis=-1)

In [ ]:
def visualize_pointclouds(gt_pointcloud, predicted_pointcloud):
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    # Ground truth point cloud
    gt_pc = o3d.geometry.PointCloud()
    gt_pc.points = o3d.utility.Vector3dVector(gt_pointcloud)
    gt_pc.paint_uniform_color([1, 0, 0])  # Red for ground truth
    vis.add_geometry(gt_pc)

    # Predicted point cloud
    pred_pc = o3d.geometry.PointCloud()
    pred_pc.points = o3d.utility.Vector3dVector(predicted_pointcloud)
    pred_pc.paint_uniform_color([0, 1, 0])  # Green for prediction
    vis.add_geometry(pred_pc)

    vis.run()
    vis.destroy_window()